In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from matplotlib import patches
from functools import reduce
import time
from PIL import Image, ImageDraw, ImageFont
from IPython.display import clear_output
import librosa
!pwd
from skimage.restoration import denoise_tv_bregman
from skimage import filters
from matplotlib import patches


/Users/raminanushiravani/Desktop/mouse_videos


In [2]:
disp = 0

In [3]:
save_path = "/Users/raminanushiravani/Desktop/mouse_videos/"
pathI = "/Users/raminanushiravani/Desktop/mouse_videos/injured/"
filesInj = os.listdir(pathI)
folInj = []
for file in filesInj:
    if '.avi' not in file:
        folInj.append(file)
        
pathH = "/Users/raminanushiravani/Desktop/mouse_videos/healthy/"
folHlt = []
filesHlt = os.listdir(pathH)
for file in filesHlt:
    if '.avi' not in file:
        folHlt.append(file)

In [ ]:
healthy_frames = {}
for fol in folHlt:
    try:
        file = os.listdir(pathH + fol)
        healthy_target_filename = pathH + fol + '/' + file[0]
        framesH = [pathH + fol + '/frames/' + f for f in os.listdir(pathH + fol + '/frames')]
        healthy_frames[healthy_target_filename] = sorted(framesH, key = lambda x: (len(x.split('/')[-1]), x))

    except: 
        x = 0
        
injured_frames = {}

for fol in folInj:
    try:
        file = os.listdir(pathI + fol)
        injured_target_filename = pathI + fol + '/' + file[0]
        framesI = [pathI + fol + '/frames/' + f for f in os.listdir(pathI + fol + '/frames')]
        injured_frames[injured_target_filename] = sorted(framesI, key = lambda x: (len(x.split('/')[-1]), x))
    except: 
        x = 0 
        


In [ ]:
from scipy import signal
from scipy.signal import argrelextrema

def extract_walking_frames(meanxs, offset):
    
    b, a = signal.butter(5, 0.2)
    y = signal.filtfilt(b, a, np.array(meanxs), padlen=10)

    local_max_idx = argrelextrema(y, np.greater)
    local_min_idx = argrelextrema(y, np.less)

    walking_frames = []
    for imin,imax in zip(local_min_idx[0],local_max_idx[0]):
        if abs(imax-imin)<10:
            walking_frames.append((min(imax+offset,imin+offset),max(imax+offset,imin+offset)))
    return walking_frames

In [ ]:
# walking or not 
batch = []
images = []
dict_video = {}
walking_frames_videos = {}
offset = 400

for hi_frames in [healthy_frames,injured_frames]:
    
    for k in hi_frames.keys():
        
        walking_frames = []
        fname = k.split('image.')[0][0:-1]
        print(fname)
        names = hi_frames[k]
        sorted_names = sorted(names, key = lambda xx : int(xx.split('/')[-1].split('.')[0]))
        count = 0

        for frame_path in sorted_names:
            
            try:
                image_data = plt.imread(frame_path)
                image_data = np.int32(np.mean(image_data,axis = -1 ))
                batch.append(image_data)
            except:
                x = 1 
                
            print(len(batch),end='\r')
            
            if len(batch) >= offset:  
                o1 = offset * count
                count += 1 
                batch_arr = np.array(batch).squeeze()
                meanxs = [] 
                kk = 1
                count_index = 0
                diff_batch = batch_arr[0:-1,:,:] - batch_arr[1:,:,:] 
                for xx in diff_batch:
                    count_index += 1 
                    mean_x = np.mean(xx[:,0:600]>0,axis=0)
                    b, a = signal.butter(8, 0.025)
                    y = signal.filtfilt(b, a, np.array(mean_x), padlen=150)
                    meanxs.append(np.argmax(y))
                    if disp:
                        figure, ax = plt.subplots(2,2,figsize=(20,10))
                        ax[0][0].imshow(xx>0)
                        ax[0][1].imshow(batch_arr[count_index])
                        
                        
                        ax[1][0].plot(np.arange(kk),np.array(meanxs))
                        ax[1][1].plot(y)

                        kk += 1 
                        plt.show()
                        time.sleep(0.001) 
                        clear_output(wait=True)


                walking_frames.append(extract_walking_frames(meanxs,o1))
                batch = []
                
        walking_frames_videos[fname] = walking_frames 

pickle.dump(walking_frames_videos , open(save_path + 'walking_frames_all_videos.pickle','wb'))        


In [ ]:

thr_merging = 30
thr_save = 50
def merge_frame_indicies(walking_frames,thr_merging):
    results = []
    count = 0 
    tuples = np.concatenate((walking_frames))

    for tuple_min_max in tuples:
        x = tuple_min_max[0]
        y = tuple_min_max[1]
        if count == 0:
            so_far = [x,y]
            count += 1
        else:
            if x - so_far[1] < thr_merging: 
                so_far[1] = y
            else:
                if so_far[1] - so_far[0] > thr_save:
                    results.append(so_far)
                count = 0
    return results

In [ ]:
# collect all frames, concatenate, merge...
new_dict = {}
for k in walking_frames_videos.keys():
    fints = []
    frame_intervals = walking_frames_videos[k]
    fints = merge_frame_indicies(frame_intervals,thr_merging)
    new_dict[k] = fints

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(new_dict,orient='index')
df.to_csv('frame_numbers.csv')

In [ ]:
# spit out a text file containing all frames in a text file for training a paw detection. 
text_file = open("mouse_walking.txt",'w')

for k in new_dict.keys():
    frames = new_dict[k]
    for frame in frames:
        for i in range(frame[0],frame[1]):
            image_file = k +'/frames/' + str(i) + '.jpg'
            text_file.write(image_file)
            text_file.write('\n')
text_file.close()

In [612]:
# Extraction
disp = 0

In [617]:
def sliding_window(image, stepSizeX,stepSizeY, windowSizeX,windowSizeY):
    for x in range(0, image.shape[1], stepSizeX):
        for y in range(0, image.shape[0], stepSizeY):
            yield (x,y , image[y:y + windowSizeY, x:x + windowSizeX])


def find_points(objs):
    thr = 0
    xys = [(2000,2000)]
    zz = []

    for obj in objs: 
        x = obj[0]
        y = obj[1]
        if abs(x - xys[-1][0])<thr or  abs(y-xys[-1][1])<thr:
            xys.append((x,y))
        else:
            xys.append((2000,2000))
            xys.append((x,y))
            
    points = []
    for xy in xys[2:]:
        if xy[0] != 2000: 
            zz.append(xy)
        elif len(zz): 
            x1 = zz[0][0]
            y1 = zz[0][-1]
            x2 = zz[-1][0]
            y2 = zz[-1][-1]
            start = (x1,y1)
            end   = (x2,y2)
            points.append((start,end))
            zz = []
    if len(zz):
        x1 = zz[0][0]
        y1 = zz[0][-1]
        x2 = zz[-1][0]
        y2 = zz[-1][-1]
        start = (x1,y1)
        end   = (x2,y2)
        points.append((start,end))
    return points


disp = 0
PROC = 0
PROCC = 0

In [620]:
def paw_mask(arr, mouse_location):
    
    stepSizeX = 5
    stepSizeY = 2
    windowSizeX = 20
    windowSizeY = 10
    area = windowSizeX * windowSizeY
    arr_mean = np.mean(arr,axis=-1)
    X = np.array([arr[:,:,0]> 190])
    Y = np.array([arr[:,:,1]> 100] and [arr[:,:,1]< 170])
    Z = np.array([arr[:,:,2]> 100] and [arr[:,:,2]< 130])
    arr_black = X*arr_mean.squeeze()
    arr_black = Y*arr_black
    arr_black = Z*arr_black
    arr_black = (arr_black == 0)*255
    arr_black = (arr_black == 255) * 0 + (arr_black == 0) * 255
    slide_ = sliding_window(arr_black.squeeze(),stepSizeX,stepSizeY,windowSizeX,windowSizeY)
    objs = []
    while True: 
        try:
            x,y , object_ = next(slide_)
            thr = np.sum(object_>1)
            if thr/area > 0.3 and y < 210 and x < 620: 
                objs.append((x,y))
        except:
            break
    rects = []
    rectangles = [] #save 

    if len(objs): 
        points = find_points(objs)
        for point in points:
            xx = point[0][0]
            yy = point[0][1]
            winX = windowSizeX
            winY = windowSizeY 
            rectangles.append(((xx,yy),(winX,winY)))
            rect = patches.Rectangle((xx,yy),winX,winY, edgecolor='b', facecolor="none")
            rects.append(rect)
    return arr_black.squeeze(), rectangles , rects

def mouse_mask(arr):
    stepSizeX = 10
    stepSizeY = 5
    windowSizeX = 50
    windowSizeY = 20
    area = windowSizeX * windowSizeY
    arr_mean = np.mean(arr,axis=-1)
    X = np.array([arr[:,:,0]> 200])#.T[None,].T
    Y = np.array([arr[:,:,1]> 200])#.T[None,].T
    Z = np.array([arr[:,:,2]> 200])#.T[None,].T
    arr_black = X*arr_mean.squeeze()
    arr_black = Y*arr_black
    arr_black = Z*arr_black
    arr_black = (arr_black == 0)*255
    slide_ = sliding_window(arr_black.squeeze(),stepSizeX,stepSizeY,windowSizeX,windowSizeY)
    objs = []
    while True: 
        try:
            x,y , object_ = next(slide_)

            thr = np.sum(object_>1)
            if thr/area > 0.9 and y < 210 and x < 620: 
                objs.append((x,y))
        except:
            break
    
    rects = []
    rectangles = [] # save 
    if len(objs): 
        points = find_points(objs)
        
        for point in points:
            xx = point[0][0]
            yy = point[0][1]

            rectangles.append(((xx,yy),(windowSizeX,windowSizeY)))
            rect = patches.Rectangle((xx,yy),windowSizeX,windowSizeY, edgecolor='r', facecolor="none")
            rects.append(rect)
            
    return arr_black.squeeze(), rectangles , rects

def detect_mouse_points(rectangles):
    xs = []
    ys = []
    for rectangle in rectangles:
        x = rectangle[0][0]
        y = rectangle[0][1]
        xs.append(x)
        ys.append(y)
    xmin = min(xs)
    ymin = min(ys)
    xmax = max(xs)+rectangle[1][0]
    ymax = max(ys)+rectangle[1][1]
    return xmin, ymin, xmax, ymax

from sklearn.cluster import KMeans

def cluster_rectangles(paw_rectangles):
    
    all_points = []
    
    for rectangle in paw_rectangles:
        x = rectangle[0][0]
        y = rectangle[0][1]
        all_points.append([x,y])
    if len(all_points)>4:
        kmeans = KMeans(n_clusters=1, random_state=0)
        kmeans1 = kmeans.fit(np.array(all_points))
        i1 = 0

        kmeans = KMeans(n_clusters=2, random_state=0)
        kmeans2 = kmeans.fit(np.array(all_points))
        i2 = min_dist(kmeans2)

        kmeans = KMeans(n_clusters=3, random_state=0)
        kmeans3 = kmeans.fit(np.array(all_points))
        i3 = min_dist(kmeans3)

        kmeans = KMeans(n_clusters=4, random_state=0)
        kmeans4 = kmeans.fit(np.array(all_points))
        i4 = min_dist(kmeans4)

        min_distances   = [i4,i3,i2,i1]
        min_index = 0
        for j in range(4):
            if min_distances[j] > 1000:
                min_index = j
                break

        kmeans_list = [kmeans4,kmeans3,kmeans2,kmeans1]

        labels = kmeans_list[min_index].predict(np.array(all_points))

        xx = np.array(all_points)[:,0]
        yy = np.array(all_points)[:,1]
        unique_classes = set(labels)
        count = 0 
        clusters = []
        for class_ in unique_classes:
            indicies = np.where(np.array(labels)==class_)[0]
            segment_x = []
            segment_y = []
            cluster = []
            for idx in indicies:
                segment_x.append(xx[idx])
                segment_y.append(yy[idx])
                cluster.append(paw_rectangles[idx])
            clusters.append(cluster)
            count+= 1 
    else: 
        return 0
    return clusters

def min_dist(kmeans_):
    xs = np.array(kmeans_.cluster_centers_)[:,0]
    ys = np.array(kmeans_.cluster_centers_)[:,1]
    distances = []
    for i in range(len(xs)):
        for j in range(len(xs)):
            if i!= j:
                distances.append(np.power((xs[i%len(xs)]-xs[(j)%len(xs)]),2) + np.power((ys[i%len(xs)]-ys[(j)%len(ys)]),2))
    return np.min(distances)

In [ ]:
filenames = open("mouse_walking.txt",'r').readlines()
frames = {}

for filename in filenames:
    
    fname = filename.split('/frames')
    if disp:
        figure, ax = plt.subplots(1,figsize=(40,40))
        
    if 1:
        
        locations = []
        
        frame = Image.open(filename.split()[0])
        arr = np.asarray(frame)
        arr_black, rectangles,rectsB = mouse_mask(arr)
        xmin, ymin, xmax, ymax = detect_mouse_points(rectangles)
        mouse_location = (xmin, ymin, xmax, ymax , 'mouse')
        
        rectMouse = patches.Rectangle((xmin,ymin),xmax-xmin,ymax-ymin, edgecolor='g', facecolor="none")
        ax.add_patch(rectMouse)
        
        locations.append(mouse_location)
        
        arr_black,paw_rectangles,rectsA = paw_mask(arr,mouse_location)
        clusters = cluster_rectangles(paw_rectangles)
        if clusters:
            for paw_rectangle in clusters:
                xmin, ymin, xmax, ymax = detect_mouse_points(paw_rectangle)
                rectPaw = patches.Rectangle((xmin,ymin),xmax-xmin,ymax-ymin, edgecolor='k', facecolor="none")
                ax.add_patch(rectPaw)
                paw_location = (xmin, ymin, xmax, ymax , 'paw')
                locations.append(paw_location)
        frames[filename.split()[0]] = locations
        
        if disp:
            figure, ax = plt.subplots(1,figsize=(40,40))
            for rect in rectsB:
                ax.add_patch(rect)
            for rect in rectsA:
                ax.add_patch(rect)
            ax.imshow(arr)
            plt.show()
            time.sleep(1)
            clear_output(wait=True)
    else: 
        print("XX " + filename)
        
    print(filename , end='\r')
    

pickle.dump(frames, open('/Users/raminanushiravani/Desktop/mouse_videos/walking_frames_w_paw_location.pickle','wb'))



/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/11.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/12.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/13.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/14.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/15.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/16.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/17.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/18.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/19.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102heal

/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/90.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/91.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/92.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/93.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/94.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/95.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/96.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/97.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/98.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102heal

/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/169.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/170.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/171.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/172.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/173.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/174.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/175.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/176.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/177.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/0

/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/303.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/304.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/305.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/306.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/307.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/308.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/309.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/310.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/311.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/0

/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/382.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/383.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/384.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/385.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/386.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/387.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/388.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/389.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/390.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/0

...

/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/743.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/744.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/745.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/746.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/747.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/748.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/749.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/750.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/751.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/0

/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/925.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/926.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/927.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/928.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/929.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/930.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/931.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/932.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/933.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/0

/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1104.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1105.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1106.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1107.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1108.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1109.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1110.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1111.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1112.jpg
/Users/raminanushiravani/Desktop/mouse_videos/

/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1182.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1183.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1184.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1185.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1186.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1187.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1188.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1189.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1190.jpg
/Users/raminanushiravani/Desktop/mouse_videos/

/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1260.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1261.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1262.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1263.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1264.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1265.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1266.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1267.jpg
/Users/raminanushiravani/Desktop/mouse_videos/healthy/036102healthy_continue_20cms_0degUP/frames/1268.jpg
/Users/raminanushiravani/Desktop/mouse_videos/

In [616]:
disp

1